### TL; DR

This notebook tries to turn all the comments in the dataframe into a dataframe where each row corresponds to their score output by detoxify. So far the the fastest option (running in 334 seconds ==5 min for 2 rows out of 214) is creating a huge dictionary by appending the dictionaries output by detoxify and using a function with split to get only the text comment 

#### Questions that I have 

Should keep like this or immediately do the mean of each feature so that in the end our dataset will be smaller? With things as they are, if each video has 150 comments, we'll have a dataset of 150x214x64 = 2 054 400 while if we keep the idea of stocking in a csv so that we need to run the thing only once, it will have size 214x64=13 696 (much easier to work with)

In [1]:
import pandas as pd
from detoxify import Detoxify
import time

In [2]:
data_folder = './Data/' # all my data is in one folder 
path_comments='comments_glipse.csv.gz'

In [3]:
df_comments = pd.read_csv(data_folder + path_comments, compression='gzip').dropna(axis=0) 
df_comments

,comments,video_id
0,"[{'id': 'UgwG1f67Ru5kL3JXmDZ4AaABAg', 'author'...",39yZGhaJay8
1,"[{'id': 'UgwDdeP0GiKI4ld_5Uh4AaABAg', 'author'...",FFm-ZpY7Fbo
2,"[{'id': 'UgySPeVqINUhtngAtbx4AaABAg', 'author'...",yfNKExa846M
3,"[{'id': 'UgyrDdhrXsWtzfv1Fbl4AaABAg', 'author'...",lCR0QTdSJfA
4,"[{'id': 'UgzHfM3U-Jx3oueGlEp4AaABAg', 'author'...",nY4EB01ldOE
...,...,...
209,"[{'id': 'UgxRhHxqrascp2Wl9XR4AaABAg', 'author'...",QAeoUItnQpE
210,"[{'id': 'UgwUu708JnCOMWgqEI54AaABAg', 'author'...",aEafDmdID70
211,"[{'id': 'UgySC3F8dJnjv0ALIW94AaABAg', 'author'...",2OVoINOkLys
212,"[{'id': 'UgwtUJyordw0yKULN6F4AaABAg', 'author'...",Ti2OUx3c6Ss


# Best thing so far, if you put as input the two first rows runs in 334 seconds

In [154]:
#has a very very small runing-time, don't think it should be optimized
def df_list(X):
    new_list_=[] #creates a list with approximately one comment per item
    n=0
    for i in X.split("'id': "):
        if n!=0: #condition to avoid taking the very first i which doesn't have anything important
            [no_keep, keep]=i.split(" 'text': ")
            [text, no_keep]=keep.split(", 'likes'")
            new_list_.append(text)
        n+=1
    return new_list_

In [155]:
df_head=df_comments.head(2)

In [160]:
#it says 193 seconds but with cached data I don't think this is the real value,
#at least not the one I had when I ran it for the first time

In [156]:
t=time.time()
toxic_df=pd.DataFrame()
dict_list={"toxicity":[],'severe_toxicity':[], 'obscene':[], 'identity_attack':[], 'insult':[], 'threat':[], 'sexual_explicit':[]}
video_id_list=[]

for i in range (len(df_head)):
    X=df_head.loc[i,"comments"]
    video_id=df_head.loc[i, "video_id"]
    new_list_=df_list(X)
    data_detoxify=Detoxify('unbiased').predict(new_list_)
    dict_list["toxicity"]+=(data_detoxify["toxicity"])
    dict_list['severe_toxicity']+=(data_detoxify['severe_toxicity'])
    dict_list['obscene']+=(data_detoxify['obscene'])
    dict_list['identity_attack']+=(data_detoxify['identity_attack'])
    dict_list['insult']+=(data_detoxify['insult'])
    dict_list['threat']+=(data_detoxify['threat'])
    dict_list['sexual_explicit']+=(data_detoxify['sexual_explicit'])
    video_id_list+=[video_id]*len(data_detoxify['toxicity'])
    
toxic_df=pd.DataFrame(data=dict_list)
toxic_df['video_id']=video_id_list
    
print('runtime=', time.time()-t)

runtime= 193.92950129508972


In [157]:
toxic_df

,toxicity,severe_toxicity,obscene,identity_attack,insult,threat,sexual_explicit,video_id
0,0.000349,0.000001,0.000025,0.000071,0.000096,0.000017,0.000013,39yZGhaJay8
1,0.000424,0.000001,0.000030,0.000061,0.000102,0.000020,0.000014,39yZGhaJay8
2,0.195656,0.000097,0.002811,0.002401,0.002793,0.000238,0.428493,39yZGhaJay8
3,0.002555,0.000007,0.000217,0.000516,0.000455,0.000046,0.000550,39yZGhaJay8
4,0.000505,0.000001,0.000037,0.000070,0.000132,0.000022,0.000015,39yZGhaJay8
...,...,...,...,...,...,...,...,...
188,0.000931,0.000002,0.000059,0.000100,0.000172,0.000027,0.000030,FFm-ZpY7Fbo
189,0.975875,0.027677,0.960910,0.006665,0.312209,0.001354,0.012716,FFm-ZpY7Fbo
190,0.001668,0.000003,0.000264,0.000098,0.000340,0.000043,0.000051,FFm-ZpY7Fbo
191,0.003152,0.000002,0.000101,0.000198,0.000643,0.000108,0.000045,FFm-ZpY7Fbo


In [158]:
t=time.time()
new_df=toxic_df.groupby(by='video_id').mean()
new_df
print('runtime=', time.time()-t)

runtime= 0.06134915351867676


# Everything else I tried so far (don't look if you don't want to have a headache)

### Turning the whole dataframe into another one 

In [4]:
#df_on=df_on.set_index(['video_id']).apply(lambda x: x.explode()).reset_index()
#command line might be usefull later

### Second idea (works but a bit naive, not optimized, long)

In [4]:
#has a very very small runing-time, don't think it should be optimized
def df_list(X):
    new_list_=[] #creates a list with approximately one comment per item
    n=0
    for i in X.split("'id': "):
        if n!=0: #condition to avoid taking the very first i which doesn't have anything important
            [no_keep, keep]=i.split(" 'text': ")
            [text, no_keep]=keep.split(", 'likes'")
            new_list_.append(text)
        n+=1
    return new_list_

In [31]:
def df_list_re(X): #does the same thing as df_list, just hoping with a better running time
    my_text=re.findall(r"(?:, 'text':)\s*(.*?)\s*(?!\1)(?:, 'likes')",X)
    return my_text

In [111]:
df_head=df_comments.head(2)

In [144]:
t=time.time()
toxic_df=pd.DataFrame()

for i in range (len(df_head)):
    X=df_head.loc[i,"comments"]
    video_id=df_head.loc[i, "video_id"]
    new_list_=df_list(X)
    data_detoxify=Detoxify('unbiased').predict(new_list_)
    temp_df=pd.DataFrame(data=data_detoxify)
    temp_df.insert(0, "video_id", video_id, False) #inserts the columns video_id as the first column in the dataset
    toxic_df = pd.concat([toxic_df,temp_df])
print('runtime=', time.time()-t)

runtime= 342.7037069797516


In [145]:
toxic_df

,video_id,toxicity,severe_toxicity,obscene,identity_attack,insult,threat,sexual_explicit
0,39yZGhaJay8,0.000349,0.000001,0.000025,0.000071,0.000096,0.000017,0.000013
1,39yZGhaJay8,0.000424,0.000001,0.000030,0.000061,0.000102,0.000020,0.000014
2,39yZGhaJay8,0.195656,0.000097,0.002811,0.002401,0.002793,0.000238,0.428493
3,39yZGhaJay8,0.002555,0.000007,0.000217,0.000516,0.000455,0.000046,0.000550
4,39yZGhaJay8,0.000505,0.000001,0.000037,0.000070,0.000132,0.000022,0.000015
...,...,...,...,...,...,...,...,...
82,FFm-ZpY7Fbo,0.000931,0.000002,0.000059,0.000100,0.000172,0.000027,0.000030
83,FFm-ZpY7Fbo,0.975875,0.027677,0.960910,0.006665,0.312209,0.001354,0.012716
84,FFm-ZpY7Fbo,0.001668,0.000003,0.000264,0.000098,0.000340,0.000043,0.000051
85,FFm-ZpY7Fbo,0.003152,0.000002,0.000101,0.000198,0.000643,0.000108,0.000045


In [9]:
t=time.time()
new_df=toxic_df.groupby(by='video_id').mean()
new_df
print('runtime=', time.time()-t)

runtime= 0.020845890045166016


### Creating our dataframe by stacking dictionnary

In [140]:
df_head=df_comments.head(2)

In [143]:
t=time.time()
toxic_df=pd.DataFrame()
dict_list={"toxicity":[],'severe_toxicity':[], 'obscene':[], 'identity_attack':[], 'insult':[], 'threat':[], 'sexual_explicit':[]}
video_id_list=[]

for i in range (len(df_head)):
    X=df_head.loc[i,"comments"]
    video_id=df_head.loc[i, "video_id"]
    new_list_=df_list(X)
    data_detoxify=Detoxify('unbiased').predict(new_list_)
    dict_list["toxicity"]+=(data_detoxify["toxicity"])
    dict_list['severe_toxicity']+=(data_detoxify['severe_toxicity'])
    dict_list['obscene']+=(data_detoxify['obscene'])
    dict_list['identity_attack']+=(data_detoxify['identity_attack'])
    dict_list['insult']+=(data_detoxify['insult'])
    dict_list['threat']+=(data_detoxify['threat'])
    dict_list['sexual_explicit']+=(data_detoxify['sexual_explicit'])
    video_id_list+=[video_id]*len(data_detoxify['toxicity'])
    
toxic_df=pd.DataFrame(data=dict_list)
toxic_df['video_id']=video_id_list
    
print('runtime=', time.time()-t)

runtime= 334.3930866718292


In [142]:
toxic_df

,toxicity,severe_toxicity,obscene,identity_attack,insult,threat,sexual_explicit,video_id
0,0.000349,0.000001,0.000025,0.000071,0.000096,0.000017,0.000013,39yZGhaJay8
1,0.000424,0.000001,0.000030,0.000061,0.000102,0.000020,0.000014,39yZGhaJay8
2,0.195656,0.000097,0.002811,0.002401,0.002793,0.000238,0.428493,39yZGhaJay8
3,0.002555,0.000007,0.000217,0.000516,0.000455,0.000046,0.000550,39yZGhaJay8
4,0.000505,0.000001,0.000037,0.000070,0.000132,0.000022,0.000015,39yZGhaJay8
...,...,...,...,...,...,...,...,...
188,0.000931,0.000002,0.000059,0.000100,0.000172,0.000027,0.000030,FFm-ZpY7Fbo
189,0.975875,0.027677,0.960910,0.006665,0.312209,0.001354,0.012716,FFm-ZpY7Fbo
190,0.001668,0.000003,0.000264,0.000098,0.000340,0.000043,0.000051,FFm-ZpY7Fbo
191,0.003152,0.000002,0.000101,0.000198,0.000643,0.000108,0.000045,FFm-ZpY7Fbo


In [ ]:
#working on dictionaries

In [151]:
from collections import Counter

In [148]:
Z=df_comments.loc[13,"comments"]
new_list_1=df_list(Z)
dict1=Detoxify('unbiased').predict(new_list_1)
dict2=data_detoxify

In [153]:
dict1 = Counter(dict1)
dict2 = Counter(dict2)
dict3 = dict(dict1 + dict2)
print (dict3)

TypeError: '>' not supported between instances of 'list' and 'int'

In [150]:
dict1+dict2

TypeError: unsupported operand type(s) for +: 'dict' and 'dict'

### Turning one line of df_comments into a dataframe

In [10]:
X=df_comments.loc[3,"comments"]
video_id=df_comments.loc[3, "video_id"]

In [11]:
t=time.time()
df_list(X)
print('runtime=', time.time()-t)

runtime= 0.0019936561584472656


In [12]:
# Turned 2 loops into one (no idea how to do it without a for loop)

new_list_=[] #creates a list with approximately one comment per item
n=0

for i in X.split("'id': "):
    if n!=0: #condition to avoid taking the very first i which doesn't have anything important
        [no_keep, keep]=i.split(" 'text': ")
        [text, no_keep]=keep.split(", 'likes'")
        new_list_.append(text)
    n+=1

In [13]:
#running time= 2 min 30

t=time.time()
data_detoxify=Detoxify('unbiased').predict(new_list_)
print('runtime=', time.time()-t)

runtime= 81.34286689758301


In [14]:
toxic_df=pd.DataFrame(data=data_detoxify)
toxic_df.insert(0, "video_id", video_id, False) #inserts the columns video_id as the first column in the dataset

In [15]:
toxic_df

,video_id,toxicity,severe_toxicity,obscene,identity_attack,insult,threat,sexual_explicit
0,lCR0QTdSJfA,0.000836,0.000002,0.000077,0.000065,0.000125,0.000073,0.000035
1,lCR0QTdSJfA,0.001311,0.000003,0.000118,0.000088,0.000234,0.000052,0.000055
2,lCR0QTdSJfA,0.000434,0.000001,0.000031,0.000075,0.000100,0.000019,0.000011
3,lCR0QTdSJfA,0.993024,0.000667,0.005575,0.005020,0.966770,0.084171,0.000864
4,lCR0QTdSJfA,0.002584,0.000003,0.000126,0.000204,0.000545,0.000101,0.000048
...,...,...,...,...,...,...,...,...
111,lCR0QTdSJfA,0.009181,0.000007,0.000251,0.001008,0.003268,0.000196,0.000050
112,lCR0QTdSJfA,0.069598,0.000053,0.002156,0.002092,0.002339,0.000084,0.117386
113,lCR0QTdSJfA,0.001757,0.000002,0.000064,0.000181,0.000272,0.000075,0.000035
114,lCR0QTdSJfA,0.045794,0.000073,0.000478,0.021208,0.005299,0.001618,0.004148


In [16]:
t=time.time()
new_df=toxic_df.groupby(by='video_id').mean()
print('runtime=', time.time()-t)

runtime= 0.002992868423461914


In [17]:
new_df

,toxicity,severe_toxicity,obscene,identity_attack,insult,threat,sexual_explicit
video_id,,,,,,,
lCR0QTdSJfA,0.208492,0.008254,0.16791,0.004402,0.086535,0.00195,0.057095


In [18]:
new_df.iloc[0,:]

toxicity           0.208492
severe_toxicity    0.008254
obscene            0.167910
identity_attack    0.004402
insult             0.086535
threat             0.001950
sexual_explicit    0.057095
Name: lCR0QTdSJfA, dtype: float64

### Improving the "spliting" part

In [19]:
import re
text = "muffins are good, cookies are bad. sauce is awesome, veggies too. fmooo mfasss, fdssaaaa."
pattern = "\.?(?P<sentence>.*?good.*?)\."
match = re.search(pattern, text)
if match != None:
    print (match.group("sentence"))

muffins are good, cookies are bad


In [5]:
X=df_comments.loc[19,"comments"]
t=time.time()
new_list=df_list(X)
print('runtime=', time.time()-t)

runtime= 0.00252532958984375


In [21]:
import re
text = X
pattern = "(?<= r'text r': \s).*(?=\s r, r'likes r')"
match = re.search(pattern, text)
if match != None:
    print (match.group("sentence"))

In [7]:
import re

In [ ]:
#do not touch !!!!!!
X=df_comments.loc[19,"comments"]
t=time.time()
my_text=re.findall(r"('text':)\s*(.*?)\s*(?!\1)(?:, 'likes')",X)
print('runtime=', time.time()-t)
#do not touch !!!!

In [27]:
X=df_comments.loc[3,"comments"]
t=time.time()
my_text=re.findall(r"(?:, 'text':)\s*(.*?)\s*(?!\1)(?:, 'likes')",X)
print('runtime=', time.time()-t)

runtime= 0.006699085235595703


In [28]:
len(my_text)

116

In [30]:
my_text[0]

"'This is one of those videos I’ll be coming back to. There is a lot of value here. A lot. Thank you. Time for me to step it up'"